In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

C:\Users\om\AppData\Roaming\Python\Python312\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\om\AppData\Roaming\Python\Python312\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\om\AppData\Roaming\Python\Python312\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update th

In [2]:
## load the train model, scaler pickel, onehot
model = load_model('model.h5')

## laod the encoder and scaler
with open('onehot_encoder_country.pkl','rb') as file:
    label_encoder_country=pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [3]:
# Example input data
input_data = {
    'credit_score':600,										
    'country': 'France',
    'gender':'Male',
    'age' : 40,
    'tenure':3,
    'balance':60000,
    'products_number':2,
    'credit_card':1,
    'active_member':1,
    'estimated_salary':50000
}

In [9]:

## Encode categorical variables 'country
country_encoded = label_encoder_country.transform([[input_data['country']]]).toarray()
country_encoded_df = pd.DataFrame(country_encoded, columns=label_encoder_country.get_feature_names_out(['country']))

C:\Users\om\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [10]:
country_encoded_df

,country_France,country_Germany,country_Spain
0,1.0,0.0,0.0


In [12]:
input_df = pd.DataFrame([input_data])
input_df

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary
0,600,France,Male,40,3,60000,2,1,1,50000


In [15]:
## Encode categorical variables
from sklearn.preprocessing import LabelEncoder

label_encoder_gender = LabelEncoder()

input_df['gender'] = label_encoder_gender.fit_transform(input_df['gender'])

input_df


,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary
0,600,France,0,40,3,60000,2,1,1,50000


In [16]:
## concatination one hot encoded
input_df= pd.concat([input_df.drop("country", axis=1), country_encoded_df], axis=1)
input_df

,credit_score,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,country_France,country_Germany,country_Spain
0,600,0,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [17]:
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516, -1.09499335,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [18]:
## Predict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step


array([[0.11169081]], dtype=float32)

In [19]:
prediction_proba = prediction[0][0]

In [20]:
prediction_proba

np.float32(0.11169081)

In [21]:
if prediction_proba > 0.5:
    print("the customer is likely to churn.")
else:
    print("the customer is not likely to churn.")

the customer is not likely to churn.
